In [1]:
import sys
import tensorflow as tf
from termcolor import colored
print(colored('Python Version: %s' % sys.version.split()[0], 'blue'))
print(colored('TensorFlow Ver: %s' % tf.__version__, 'magenta'))

C:\Users\mikeb\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Python Version: 3.6.2
TensorFlow Ver: 1.6.0


In [2]:
n_epoch = int(input('Enter no. of epochs for RNN training: '))

Enter no. of epochs for RNN training: 90


In [3]:
print(colored('No. of epochs: %d' % n_epoch, 'red'))

No. of epochs: 90


In [4]:
import pandas as pd

In [5]:
data = pd.read_csv('fraud_training_set.csv')
data.head(10)

,Unnamed: 0,fake/real,reviews
0,0,1,We stayed 2 nights over spring break in what w...
1,1,1,Stayed at the Fitzpatrick as a result of all o...
2,2,1,We booked this hotel as a last minute vacation...
3,3,1,This hotel is a shambles-furniture literally f...
4,4,1,The hotel itself was beautiful and wonderful s...
5,5,1,"I'd been searching for a cool, non-chain hotel..."
6,6,1,I stayed @ Affinia for one night on a purpose ...
7,7,1,Having stayed at the excellent Affinia chain i...
8,8,1,I've been here for 4 days. Great location righ...
9,9,1,"Stayed at the Fitzpatrick before, about 4 year..."


In [6]:
print('data.shape:', data.shape)

data.shape: (1600, 3)


In [7]:
data["fake/real"].value_counts()

1    800
0    800
Name: fake/real, dtype: int64

In [8]:
X = data.reviews
y = data["fake/real"]

In [9]:
X.head(10)

0    We stayed 2 nights over spring break in what w...
1    Stayed at the Fitzpatrick as a result of all o...
2    We booked this hotel as a last minute vacation...
3    This hotel is a shambles-furniture literally f...
4    The hotel itself was beautiful and wonderful s...
5    I'd been searching for a cool, non-chain hotel...
6    I stayed @ Affinia for one night on a purpose ...
7    Having stayed at the excellent Affinia chain i...
8    I've been here for 4 days. Great location righ...
9    Stayed at the Fitzpatrick before, about 4 year...
Name: reviews, dtype: object

In [10]:
y.head(10)

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: fake/real, dtype: int64

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [12]:
vect = TfidfVectorizer(stop_words='english', token_pattern=r'\b\w{2,}\b')
dummy = DummyClassifier(strategy='most_frequent', random_state=0)

dummy_pipeline = make_pipeline(vect, dummy)

In [13]:
dummy_pipeline.named_steps

{'dummyclassifier': DummyClassifier(constant=None, random_state=0, strategy='most_frequent'),
 'tfidfvectorizer': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
         stop_words='english', strip_accents=None, sublinear_tf=False,
         token_pattern='\\b\\w{2,}\\b', tokenizer=None, use_idf=True,
         vocabulary=None)}

In [14]:
# Cross Validation
cv = cross_val_score(dummy_pipeline, X, y, scoring='accuracy', cv=10, n_jobs=-1)
print(colored('\nDummy Classifier\'s Accuracy: %0.5f\n' % cv.mean(), 'yellow'))


Dummy Classifier's Accuracy: 0.50000



In [15]:

from sklearn.naive_bayes import MultinomialNB

In [16]:
vect = TfidfVectorizer(stop_words='english', 
                       token_pattern=r'\b\w{2,}\b',
                       min_df=1, max_df=0.1,
                       ngram_range=(1,2))
mnb = MultinomialNB(alpha=2)

mnb_pipeline = make_pipeline(vect, mnb)

In [17]:
mnb_pipeline.named_steps

{'multinomialnb': MultinomialNB(alpha=2, class_prior=None, fit_prior=True),
 'tfidfvectorizer': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=0.1, max_features=None, min_df=1,
         ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
         stop_words='english', strip_accents=None, sublinear_tf=False,
         token_pattern='\\b\\w{2,}\\b', tokenizer=None, use_idf=True,
         vocabulary=None)}

In [18]:
# Cross Validation
cv = cross_val_score(mnb_pipeline, X, y, scoring='accuracy', cv=10, n_jobs=-1)
print(colored('\nMultinomialNB Classifier\'s Accuracy: %0.5f\n' % cv.mean(), 'green'))


MultinomialNB Classifier's Accuracy: 0.86000



In [19]:
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
from tflearn.datasets import imdb

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(1,1), token_pattern=r'\b\w{1,}\b')

In [22]:
vect.fit(X_train)
vocab = vect.vocabulary_

In [23]:
def convert_X_to_X_word_ids(X):
    return X.apply( lambda x: [vocab[w] for w in [w.lower().strip() for w in x.split()] if w in vocab] )

In [24]:
X_train_word_ids = convert_X_to_X_word_ids(X_train)
X_test_word_ids  = convert_X_to_X_word_ids(X_test)

In [25]:
X_train.head()

1379    As a frequent traveler for both business and p...
938     A few weeks ago we stayed at the Hard Rock Hot...
1197    I'd expect a "luxury" hotel to pay more attent...
730     I stayed for just one nite at the Sheraton Hot...
1145    After arriving at the Sofitel Chicago Water To...
Name: reviews, dtype: object

In [26]:
X_train_word_ids.head()

1379    [753, 287, 3485, 8367, 3415, 1232, 1374, 591, ...
938     [287, 3255, 8932, 484, 8907, 7709, 804, 8126, ...
1197    [3099, 287, 4087, 8252, 5871, 5287, 831, 8252,...
730     [4152, 7709, 3415, 4536, 5625, 5461, 804, 8126...
1145    [464, 741, 804, 8126, 7496, 1626, 8890, 8317, ...
Name: reviews, dtype: object

In [27]:
print('X_train_word_ids.shape:', X_train_word_ids.shape)
print('X_test_word_ids.shape:', X_test_word_ids.shape)

X_train_word_ids.shape: (1440,)
X_test_word_ids.shape: (160,)


In [28]:
X_train_padded_seqs = pad_sequences(X_train_word_ids, maxlen=20, value=0)
X_test_padded_seqs  = pad_sequences(X_test_word_ids , maxlen=20, value=0)

In [29]:
print('X_train_padded_seqs.shape:', X_train_padded_seqs.shape)
print('X_test_padded_seqs.shape:', X_test_padded_seqs.shape)

X_train_padded_seqs.shape: (1440, 20)
X_test_padded_seqs.shape: (160, 20)


In [30]:
pd.DataFrame(X_train_padded_seqs).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,753,287,3485,8367,3415,1232,1374,591,1703,591,1810,331,804,287,6449,6200,706,4211,8252,4036
1,287,3255,8932,484,8907,7709,804,8126,3878,6834,4087,4228,8126,4087,8870,287,1376,3509,8126,3615
2,3099,287,4087,8252,5871,5287,831,8252,8126,6859,6444,1039,8178,4152,3455,3831,8520,8126,8126,5218
3,4152,7709,3415,4536,5625,5461,804,8126,7209,4087,591,8319,591,8870,287,4811,5414,464,6434,287
4,464,741,804,8126,7496,1626,8890,8317,4087,4152,8870,3754,9027,6893,591,5347,6859,6688,3823,1039


In [31]:
pd.DataFrame(X_test_padded_seqs).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,7477,5347,5073,5696,8520,8126,5582,2510,804,8126,1642,8650,737,3912,8079,5073,4009,1256,706,8889
1,8907,7709,804,8126,6810,1472,8466,8932,591,4228,1848,8126,7496,2636,5501,1805,9027,8126,2295,8126
2,4152,1412,8126,6688,4782,8252,1211,287,6859,804,8126,1052,5582,2315,111,3415,287,6412,9027,287
3,8174,4451,287,8300,5625,8252,753,8126,7678,6417,8947,3509,14,7678,8252,218,591,3823,8252,3269
4,4152,8870,6444,3068,8965,8907,1212,287,5443,804,8126,3878,6834,4087,4228,4458,8870,382,1106,5693


In [32]:
unique_y_labels = list(y_train.value_counts().index)
unique_y_labels

[0, 1]

In [33]:
len(unique_y_labels)

2

In [34]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(unique_y_labels)

LabelEncoder()

In [35]:

print('')
for label_id, label_name in zip(le.transform(unique_y_labels), unique_y_labels):
    print('%d: %s' % (label_id, label_name))
print('')


0: 0
1: 1



In [36]:
y_train = to_categorical(y_train.map(lambda x: le.transform([x])[0]), nb_classes=len(unique_y_labels))
y_test  = to_categorical(y_test.map(lambda x:  le.transform([x])[0]), nb_classes=len(unique_y_labels))

In [37]:
y_train[0:3]

array([[1., 0.],
       [1., 0.],
       [1., 0.]])

In [38]:
print('y_train.shape:', y_train.shape)
print('y_test.shape:', y_test.shape)

y_train.shape: (1440, 2)
y_test.shape: (160, 2)


In [39]:
size_of_each_vector = X_train_padded_seqs.shape[1]
vocab_size = len(vocab)
no_of_unique_y_labels = len(unique_y_labels)

In [40]:
print('size_of_each_vector:', size_of_each_vector)
print('vocab_size:', vocab_size)
print('no_of_unique_y_labels:', no_of_unique_y_labels)

size_of_each_vector: 20
vocab_size: 9150
no_of_unique_y_labels: 2


In [41]:

#sgd = tflearn.SGD(learning_rate=1e-4, lr_decay=0.96, decay_step=1000)

net = tflearn.input_data([None, size_of_each_vector]) # The first element is the "batch size" which we set to "None"
net = tflearn.embedding(net, input_dim=vocab_size, output_dim=128) # input_dim: vocabulary size
net = tflearn.lstm(net, 128, dropout=0.6) # Set the dropout to 0.6
net = tflearn.fully_connected(net, no_of_unique_y_labels, activation='softmax') # relu or softmax
net = tflearn.regression(net, 
                         optimizer='adam',  # adam or ada or adagrad # sgd
                         learning_rate=1e-4,
                         loss='categorical_crossentropy')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [42]:
#model = tflearn.DNN(net, tensorboard_verbose=0, checkpoint_path='SavedModels/model.tfl.ckpt')
model = tflearn.DNN(net, tensorboard_verbose=0)

In [43]:
model.fit(X_train_padded_seqs, y_train, 
          validation_set=(X_test_padded_seqs, y_test), 
          n_epoch=n_epoch,
          show_metric=True, 
          batch_size=400)

Training Step: 359  | total loss: 0.35994 | time: 0.270s
| Adam | epoch: 090 | loss: 0.35994 - acc: 0.8745 -- iter: 1200/1440
Training Step: 360  | total loss: 0.34674 | time: 1.334s
| Adam | epoch: 090 | loss: 0.34674 - acc: 0.8799 | val_loss: 0.60043 - val_acc: 0.7375 -- iter: 1440/1440
--


In [44]:
model.save('SavedModels/model10.tfl')
print(colored('Model Saved!', 'red'))

INFO:tensorflow:C:\Users\mikeb\Desktop\Review_Fraud_Detector\SavedModels\model10.tfl is not in all_model_checkpoint_paths. Manually adding it.
Model Saved!


In [45]:
model.load('SavedModels/model10.tfl')
print(colored('Model Loaded!', 'red'))

INFO:tensorflow:Restoring parameters from C:\Users\mikeb\Desktop\Review_Fraud_Detector\SavedModels\model10.tfl
Model Loaded!


In [46]:

import numpy as np
from sklearn import metrics

In [47]:
pred_classes = [np.argmax(i) for i in model.predict(X_test_padded_seqs)]
true_classes = [np.argmax(i) for i in y_test]

print(colored('\nRNN Classifier\'s Accuracy: %0.5f\n' % metrics.accuracy_score(true_classes, pred_classes), 'cyan'))


RNN Classifier's Accuracy: 0.73750



In [48]:
ids_of_titles = range(0,200) # range(X_test.shape[0]) 

for i in ids_of_titles:
    pred_class = np.argmax(model.predict([X_test_padded_seqs[i]]))
    true_class = np.argmax(y_test[i])
    
    print(X_test.values[i])
    print('pred_class:', le.inverse_transform(pred_class))
    print('true_class:', le.inverse_transform(true_class))
    print('')

so my sweetie lures me out under the guise of dinner at the China Grill, upon arrival he tells me his boys are watching the game in a suite upstairs and want to say hi...we arrive at the 35th floor and my oh my!!! SURPRISE!!! ladies and gentlemen THIS is something to be admired...spacious and roomy, yet warm, cozy and inviting...great bed, comfortable furniture, great 42 inch plasmas, and the views are UNBELIEVABLE...my only complaint? a shower should have a door to keep the heat IN...other than that no complaints, room service was great, morning coffee was yummo...overall a fabulous experience!!! 

pred_class: 1
true_class: 1

We stayed at the Ritz Carlton two weeks prior, and in comparison the Sofitel does not come close. With the deal the Ritz Carlton is putting on right now, stay two nights...get one free, choose the nicer option! Pros: Room Service: Quick! 15 minutes tops. Bathroom: Large bathroom with a separate shower and tub. Lobby: Very impressive, and the doormen are on top o

C:\Users\mikeb\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\mikeb\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\mikeb\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\mikeb\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

pred_class: 1
true_class: 1

Swissotel was the cleanest hotel I have ever stayed in! The room and bathroom were quite large for downtown Chicago. The pool and hot tub were also very nice. I would definately recommend this hotel. We didn't hear any noise in our room from other guests or from the city. It is in a great location - walking distance to Millenium Park, the Loop and Michigan Ave. 

pred_class: 0
true_class: 1

I was surprised by the fact that there were no extra sheets or blankets to make up the hide a bed. I called the desk and asked for some and they said that I couldn't have any and that only the maid could come and set up the bed. My husband was ill and I wanted the bed set up so I wouldn't disturb him. but they were un moved and wouldn't bring the sheets. I ended up sleeping on the mattress with bath towels over it and another bath towel as a blanket. They could have given me a set of sheets and a blanket, I didn't pay that much a night to sleep on a bare mattress. Serio

C:\Users\mikeb\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\mikeb\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\mikeb\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\mikeb\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

pred_class: 0
true_class: 0

My daughter and I woke in the morning wanting to go swimming. When we arrived at the pool the water was covered by a white scum. I then attempted to use both of the phones at the pool, one white phone and one emergency red phone, to call the desk. Both were out of service!!!! I am glad there wasn't an emergency. As we were exited the pool area I ran into a hotel employee and told her about the problems and then asked her to call us when the pool was clean.... never heard back. One more thing, in our room the furniture was mostly child friendly except for a large metal and glass outdated coffee table. Within 30 minutes my 5 year old fell and hit her mouth and began bleeding. They need to update this piece of furniture.

pred_class: 1
true_class: 1



C:\Users\mikeb\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\mikeb\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\mikeb\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\mikeb\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

IndexError: index 160 is out of bounds for axis 0 with size 160